In [14]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"

driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

In [ ]:
import pandas as pd
import numpy as np
df_triplet = pd.read_csv('data/all_triplet_lang_result.csv')
df_entity = pd.read_csv('output/entity_llm_embedding.csv')
df_entity['embedding'] = df_entity.embedding.apply(eval).apply(np.array)

selected = df_triplet[df_triplet['id'] == 'c736c675-d1fe-422d-ba3f-16dd54cdf477']

entity_embedding = []
for index, row in selected.iterrows():
    subject = df_entity[df_entity['entity'] == row['subject']]['embedding']
    predicate = df_entity[df_entity['entity'] == row['predicate']]['embedding']
    object = df_entity[df_entity['entity'] == row['object']]['embedding']
    entity_embedding.append([subject.values[0], predicate.values[0], object.values[0]])

average_embedding = np.mean(entity_embedding, axis=0)

array([[-0.00826139, -0.01771853,  0.01101754, ..., -0.01142988,
        -0.0096408 , -0.0121578 ],
       [-0.01736911, -0.01399937,  0.00449301, ..., -0.01365874,
        -0.01284346, -0.01121555],
       [-0.01111349, -0.01041773, -0.01105122, ..., -0.01079431,
         0.00477596, -0.01097073]])

In [ ]:
records, summary, key = driver.execute_query("""
MATCH (s)-[:RELATED_TO]->(p:VERB)-[:RELATED_TO]->(o)                                        
MATCH (i:Requirement)-[:CONTAINS]->(s)
MATCH (i:Requirement)-[:CONTAINS]->(p)
MATCH (i:Requirement)-[:CONTAINS]->(o)
                                             
WITH i.id as id, i.sentence as sentence, vector.similarity.cosine(s.embedding, $embed_one) as subjectScore, 
    vector.similarity.cosine(o.embedding, $embed_three) as objectScore,
    vector.similarity.cosine(p.embedding, $embed_two) as verbScore
RETURN max(id) as id, sentence, AVG((subjectScore + objectScore + verbScore)/3) as score
ORDER BY score DESC LIMIT 10
""", embed_one=average_embedding[0], embed_two=average_embedding[1], embed_three=average_embedding[2])

for row in records:
    print(row['id'], row['sentence'], row['score'])

c736c675-d1fe-422d-ba3f-16dd54cdf477 Track every single unit of your product and set alerts to be notified when the product runs low on stock. 0.9699025286568536
14169e42-3c8c-4554-8143-b871d94ac741 Easily migrate products from an external system to quickly start taking orders and bring in revenue. 0.9461250901222229
4afcc1d0-99e4-4db2-8e7f-2f2d80206fb2 The system shall update the stock to reflect the new quantity of the added product. 0.9448814656999376
01379659-6e85-4055-a25b-c57056cbe842 The system must verify that the product associated with the coupon is available in stock before applying the discount. 0.9410846432050068
bc249626-1744-4ff0-9a29-39a1c3ba76a2 The system must allow the user to input the amount to be paid and the cash received from the customer. 0.9375385642051697
b6a796f7-64f5-4cb6-ba5a-736cca8ef440 The system shall permit the Sales-Assistant to enter the quantity for the selected product if the customer purchases more than the minimum wholesale quantity defined in t